<a href="https://colab.research.google.com/github/Mini-project4ppl/Major_Project/blob/main/DR_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Define paths to input and output directories
# input_dir = 'C:\\Users\\addep\\OneDrive\\Documents\\diabetic-retinopathy-unziped\\main train\\main train' #dataset path
input_dir= "C:\\Users\\addep\\OneDrive\\Documents\\DR" #path of dataset containg subfolders

In [ ]:
# Define CLAHE parameters
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))

c=0

# Loop through each class directory in the input directory
for class_dir in os.listdir(input_dir):
    input_class_path = os.path.join(input_dir, class_dir)

    # Loop through each image in the class directory
    for img_name in os.listdir(input_class_path):

        # Read image
        img_path = os.path.join(input_class_path, img_name)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        # Apply CLAHE
        img_clahe = clahe.apply(img)

        # Save CLAHE enhanced image, replacing the existing file
        img_path_clahe = img_path  # Use the same file path
        cv2.imwrite(img_path_clahe, img_clahe)

    print("Level",c,"done")
    c=c+1

print("Preprocessing complete.")


In [ ]:
#DCGAN

#GAN

import torch
import torch.nn as nn
import torch.nn.functional as F


def weights_init(w):
    """
    Initializes the weights of the layer, w.
    """
    classname = w.__class__.__name__
    if classname.find('conv') != -1:
        nn.init.normal_(w.weight.data, 0.0, 0.02)
    elif classname.find('bn') != -1:
        nn.init.normal_(w.weight.data, 1.0, 0.02)
        nn.init.constant_(w.bias.data, 0)

# Define the Generator Network
class Generator(nn.Module):
    def __init__(self, params):
        super().__init__()

        # Input is the latent vector Z.
        self.tconv1 = nn.ConvTranspose2d(params['nz'], params['ngf']*8*2,
            kernel_size=5, stride=1, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(params['ngf']*8*2)

        # Input Dimension: (ngf*8) x 5 x 5
        self.tconv2 = nn.ConvTranspose2d(params['ngf']*8*2, params['ngf']*4*2,
            5, 3, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(params['ngf']*4*2)

        # Input Dimension: (ngf*4) x 15 x 15
        self.tconv3 = nn.ConvTranspose2d(params['ngf']*4*2, params['ngf']*2*2,
            5, 3, 1, bias=False)
        self.bn3 = nn.BatchNorm2d(params['ngf']*2*2)

        # Input Dimension: (ngf*2) x 45 x 45
        self.tconv4 = nn.ConvTranspose2d(params['ngf']*2*2, params['ngf']*2,
            4, 2, 1, bias=False)
        self.bn4 = nn.BatchNorm2d(params['ngf']*2)

        # Input Dimension: (ngf*2) x 90 x 90
        self.tconv5 = nn.ConvTranspose2d(params['ngf']*2, params['ngf'],
            4, 2, 1, bias=False)
        self.bn5 = nn.BatchNorm2d(params['ngf'])

        # # Input Dimension: (ngf*2) x 180 x 180
        # self.tconv6 = nn.ConvTranspose2d(params['ngf']*2, params['ngf'],
        #     4, 2, 1, bias=False)
        # self.bn6 = nn.BatchNorm2d(params['ngf'])

        # Input Dimension: (ngf*2) x 180 x 180
        self.tconv7 = nn.ConvTranspose2d(params['ngf'], params['nc'],
            4, 2, 1, bias=False)
        #Output Dimension: (nc) 360 x 360
    def forward(self, x):
        x = F.relu(self.bn1(self.tconv1(x)))
        x = F.relu(self.bn2(self.tconv2(x)))
        x = F.relu(self.bn3(self.tconv3(x)))
        x = F.relu(self.bn4(self.tconv4(x)))
        x = F.relu(self.bn5(self.tconv5(x)))
        # x = F.relu(self.bn6(self.tconv6(x)))


        x = F.tanh(self.tconv7(x))

        return x

# Define the Discriminator Network
class Discriminator(nn.Module):
    def __init__(self, params):
        super().__init__()

        # Input Dimension: (nc) x 360 x 360
        self.conv1 = nn.Conv2d(params['nc'], params['ndf'],
            4, 2, 1, bias=False)

        # Input Dimension: (ndf) x 180 x 180
        self.conv2 = nn.Conv2d(params['ndf'], params['ndf']*2,
            4, 2, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(params['ndf']*2)

        # Input Dimension: (ndf*2) x 90 x 90
        self.conv3 = nn.Conv2d(params['ndf']*2, params['ndf']*4,
            4, 2, 1, bias=False)
        self.bn3 = nn.BatchNorm2d(params['ndf']*4)


        # Input Dimension: (ndf*8) x 45 x 45
        self.conv4 = nn.Conv2d(params['ndf']*4, params['ndf']*8,
            5, 3, 1, bias=False)
        self.bn4 = nn.BatchNorm2d(params['ndf']*8)
        # Output Dimesnion 15 x 15

        # Input Dimension: (ndf*8) x 15 x 15
        self.conv5 = nn.Conv2d(params['ndf']*8, params['ndf']*16,
            5, 3, 1, bias=False)
        self.bn5 = nn.BatchNorm2d(params['ndf']*16)
        # Output Dimesnion 5 x 5


        # Input 5 x 5
        self.conv6 = nn.Conv2d(params['ndf']*16, 1, 5, 1, 0, bias=False)
        # OUtput 1 x 1
        # self.bn5 = nn.BatchNorm2d(params['ndf']*1)


    def forward(self, x):
        x = F.leaky_relu(self.conv1(x), 0.2, True)
        x = F.leaky_relu(self.bn2(self.conv2(x)), 0.2, True)
        x = F.leaky_relu(self.bn3(self.conv3(x)), 0.2, True)
        x = F.leaky_relu(self.bn4(self.conv4(x)), 0.2, True)
        x = F.leaky_relu(self.bn5(self.conv5(x)), 0.2, True)

        x = F.sigmoid(self.conv6(x))

        return x


import torch
import torchvision.transforms as transforms
import torchvision.datasets as dset

# Directory containing the data on your local machine.
root = "C:\\Users\\addep\\OneDrive\\Documents\\DR"

def get_dataset(params):
    """
    Loads the dataset and applies preprocessing steps to it.
    Returns a PyTorch DataLoader.

    """
    # Data preprocessing.
    transform = transforms.Compose([
        transforms.Resize(params['imsize']),
        transforms.CenterCrop(params['imsize']),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    # Create the dataset.
    dataset = dset.ImageFolder(root=root, transform=transform)

    # Create the dataloader.
    dataloader = torch.utils.data.DataLoader(dataset,
                                             batch_size=params['bsize'],
                                             shuffle=True,
                                             num_workers=0)  # Change this to the number of CPU cores available on your local machine, or keep it as 0 if you don't want to use multiprocessing.

    return dataloader


In [ ]:
#DCGAN Train

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.utils as vutils
import numpy as np
import random

# Directory where you want to save the model checkpoints.
model_dir = 'C:\\Users\\akura\\OneDrive\\Documents\\dcgan_model.pth'

# Set random seed for reproducibility.
seed = 369
random.seed(seed)
torch.manual_seed(seed)
print("Random Seed: ", seed)

# Parameters to define the model.
params = {
    "bsize": 2,  # Batch size during training.
    'imsize': 360,  # Spatial size of training images. All images will be resized to this size during preprocessing.
    'nc': 3,  # Number of channels in the training images. For colored images, this is 3.
    'nz': 100,  # Size of the Z latent vector (the input to the generator).
    'ngf': 32,  # Size of feature maps in the generator. The depth will be multiples of this.
    'ndf': 64,  # Size of feature maps in the discriminator. The depth will be multiples of this.
    'nepochs': 10,  # Number of training epochs.
    'lr': 0.0002,  # Learning rate for optimizers.
    'beta1': 0.5,  # Beta1 hyperparameter for Adam optimizer.
    'save_epoch': 2  # Save step.
}

# Use GPU if available, else use CPU.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device, " will be used.\n")

# Get the data.
dataloader = get_dataset(params)

# Create the generator.
netG = Generator(params).to(device)
netG.apply(weights_init)
print(netG)

# Create the discriminator.
netD = Discriminator(params).to(device)
netD.apply(weights_init)
netD.add_module('sigmoid', nn.Sigmoid())
print(netD)

# Binary Cross Entropy loss function.
criterion = nn.BCELoss()

fixed_noise = torch.randn(64, params['nz'], 1, 1, device=device)

real_label = 1
fake_label = 0

# Optimizer for the discriminator.
optimizerD = optim.Adam(netD.parameters(), lr=params['lr'], betas=(params['beta1'], 0.999))
# Optimizer for the generator.
optimizerG = optim.Adam(netG.parameters(), lr=params['lr'], betas=(params['beta1'], 0.999))

# Stores generated images as training progresses.
img_list = []
# Stores generator losses during training.
G_losses = []
# Stores discriminator losses during training.
D_losses = []

iters = 0

print("Starting Training Loop...")
print("-" * 25)

for epoch in range(params['nepochs']):
    batches_done = 0
    for i, data in enumerate(dataloader, 0):
        batches_done += 1
        real_data = data[0].to(device)
        b_size = real_data.size(0)

        netD.zero_grad()
        label = torch.full((b_size,), real_label, device=device).float()  # Cast to float
        output = netD(real_data).view(-1)
        errD_real = criterion(output, label)
        errD_real.backward()
        D_x = output.mean().item()

        noise = torch.randn(b_size, params['nz'], 1, 1, device=device)
        fake_data = netG(noise)
        label.fill_(fake_label)
        output = netD(fake_data.detach()).view(-1)
        errD_fake = criterion(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()

        errD = errD_real + errD_fake
        optimizerD.step()

        netG.zero_grad()
        label.fill_(real_label)
        output = netD(fake_data).view(-1)
        errG = criterion(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()

        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, params['nepochs'], i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        G_losses.append(errG.item())
        D_losses.append(errD.item())

        if (iters % 100 == 0) or ((epoch == params['nepochs'] - 1) and (i == len(dataloader) - 1)):
            with torch.no_grad():
                fake_data = netG(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake_data, padding=2, normalize=True))

        iters += 1
        print("Epoch : {}, Batch : {}".format(epoch, batches_done))

    if epoch % params['save_epoch'] == 0:
        torch.save({
            'generator': netG.state_dict(),
            'discriminator': netD.state_dict(),
            'optimizerG': optimizerG.state_dict(),
            'optimizerD': optimizerD.state_dict(),
            'params': params
        }, 'model_epoch_{}.pth'.format(epoch))

torch.save({
    'generator': netG.state_dict(),
    'discriminator': netD.state_dict(),
    'optimizerG': optimizerG.state_dict(),
    'optimizerD': optimizerD.state_dict(),
    'params': params
}, 'model_final.pth')


In [ ]:
#DCGAN Generate

import os
import torch
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt

# Load the checkpoint file.
path = "C:\\Users\\addep\\OneDrive\\Documents\\model_final.pth"
state_dict = torch.load(path, map_location='cpu')

# Set the device to run on: GPU or CPU.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Get the 'params' dictionary from the loaded state_dict.
params = state_dict['params']

# Create the generator network.
netG = Generator(params).to(device)
# Load the trained generator weights.
netG.load_state_dict(state_dict['generator'])
print(netG)

# Set the number of generated outputs
num_output = 1000

# Path to the test dataset with 5 subfolders.
test_dataset_path = "C:\\Users\\addep\\OneDrive\\Documents\\Test"

# Create a new dataset with 5 subfolders for generated images.
output_dataset_path = "C:\\Users\\addep\\OneDrive\\Documents\\GAN"
os.makedirs(output_dataset_path, exist_ok=True)

# Generate images for each subfolder in the test dataset.
for subfolder_name in os.listdir(test_dataset_path):
    subfolder_path = os.path.join(test_dataset_path, subfolder_name)

    # Create subfolder in the generated dataset.
    output_subfolder_path = os.path.join(output_dataset_path, subfolder_name)
    os.makedirs(output_subfolder_path, exist_ok=True)

    # Get latent vector Z from unit normal distribution.
    noise = torch.randn(num_output, params['nz'], 1, 1, device=device)

    # Turn off gradient calculation to speed up the process.
    with torch.no_grad():
        # Get generated image from the noise vector using the trained generator.
        generated_img = netG(noise).detach().cpu()

    # Save generated images to the corresponding subfolder.
    for i in range(num_output):
        img_filename = f"pic_generated_{subfolder_name}_{i}.jpg"
        img_path = os.path.join(output_subfolder_path, img_filename)

        # Convert PyTorch tensor to NumPy array and normalize values
        img_np = np.transpose(vutils.make_grid(generated_img[i], padding=2, normalize=True).numpy(), (1, 2, 0))
        plt.imsave(img_path, img_np)

print("Generated images saved to:", output_dataset_path)



In [ ]:
#training DenseNet-201 model

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import torchvision.models as models
from torch.nn.parallel import DataParallel

# Set the path to your dataset
data_dir = 'C:\\Users\\addep\\OneDrive\\Documents\\DR'
# Define the transformations applied to the input images
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the dataset
dataset = ImageFolder(data_dir, transform=transform)

# Define the DataLoader
batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Load the pre-trained DenseNet201 model
model = models.densenet201(pretrained=True)

# Replace the classifier to match the number of classes in your dataset
num_classes = len(dataset.classes)
in_features = model.classifier.in_features
model.classifier = nn.Linear(in_features, num_classes)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Check if multiple GPUs are available and move the model to GPU if possible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs for data parallelism.")
    model = DataParallel(model)
model = model.to(device)

# Training loop
num_epochs = 20

for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in dataloader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

# Save the trained model to your desired location
save_dir = "C:\\Users\\addep\\OneDrive\\Documents"
model_name = "Densenet.pth"
model_path = os.path.join(save_dir, model_name)
torch.save(model.state_dict(), model_path)


In [ ]:
# Train AlexNet

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Set the paths for your local system
train_data_dir = 'C:\\Users\\addep\\OneDrive\\Documents\\DR'  # Adjust this path

batch_size = 16
epochs = 20
input_shape = (224, 224, 3)  # Adjust the input size based on your dataset
num_classes = 5  # Number of severity classes

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical')

model = Sequential()

model.add(Conv2D(96, (11, 11), strides=(4, 4), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D((3, 3), strides=(2, 2)))

model.add(Conv2D(256, (5, 5), activation='relu'))
model.add(MaxPooling2D((3, 3), strides=(2, 2)))

model.add(Conv2D(384, (3, 3), activation='relu'))
model.add(Conv2D(384, (3, 3), activation='relu'))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((3, 3), strides=(2, 2)))

model.add(Flatten())

model.add(Dense(4096, activation='relu'))
model.add(Dense(4096, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# No validation_generator and validation_data in the fit method
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    callbacks=[ModelCheckpoint('fundus_severity_model.h5', monitor='accuracy', save_best_only=True, mode='max', verbose=1),
               EarlyStopping(monitor='accuracy', patience=10, mode='max', verbose=1)])

# Save the trained model
model.save('Alexnet.h5')

import os

# Save the trained model to your desired location
save_dir = "C:\\Users\\addep\\OneDrive\\Documents"
model_name = "Alexnet_trail.h5"
model_path = os.path.join(save_dir, model_name)
model.save(model_path)

In [ ]:
#Convert .h5 to .pth

from keras.models import load_model

keras_model = load_model('C:\\Users\\addep\\OneDrive\\Documents\\Alexnet_trail.h5')

import torch
import torch.nn as nn

# Define your PyTorch model class
class PyTorchModel(nn.Module):
    def __init__(self):
        super(PyTorchModel, self).__init__()
        # Define your layers here

    def forward(self, x):
        # Define the forward pass of your model
        return x

# Create an instance of your PyTorch model
pytorch_model = PyTorchModel()


# Iterate over layers and transfer weights
for keras_layer, pytorch_layer in zip(keras_model.layers, pytorch_model.modules()):
    if isinstance(pytorch_layer, nn.Conv2d) or isinstance(pytorch_layer, nn.Linear):
        # Transfer weights
        pytorch_layer.weight.data = torch.tensor(keras_layer.get_weights()[0])
        pytorch_layer.bias.data = torch.tensor(keras_layer.get_weights()[1])


torch.save(pytorch_model.state_dict(), 'C:\\Users\\addep\\OneDrive\\Documents\\Alexnet_trail.pth')

In [ ]:
#Densenet accuracy
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.models as models

# Set the path to the folder containing test images
test_data_dir = 'C:\\Users\\addep\\OneDrive\\Documents\\Test'

# Define the transformations applied to the input images
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the test dataset
test_dataset = ImageFolder(test_data_dir, transform=transform)

# Create the DataLoader for the test data
batch_size = 64
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Load the pre-trained DenseNet model
dn_model_path = "C:\\Users\\addep\\OneDrive\\Documents\\Densenet.pth"
dn_model = models.densenet201(pretrained=True)
dn_model.classifier = nn.Linear(dn_model.classifier.in_features, len(test_dataset.classes))
dn_model.load_state_dict(torch.load(dn_model_path), strict=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dn_model.to(device)
dn_model.eval()

# Evaluation function
def evaluate_model(model, dataloader):
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# Evaluate the DenseNet model on the test dataset
dn_model_accuracy = evaluate_model(dn_model, test_dataloader)

# Print the accuracy
print(f"Accuracy for DenseNet Model: {dn_model_accuracy:.2f}%")


In [ ]:
#Alex Net Accuracy

import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.models as models

# Set the path to the folder containing test images
test_data_dir = 'C:\\Users\\addep\\OneDrive\\Documents\\Test'

# Define the transformations applied to the input images
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the test dataset
test_dataset = ImageFolder(test_data_dir, transform=transform)

# Create the DataLoader for the test data
batch_size = 16
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Load the pre-trained AlexNet model
alexnet_model_path = 'C:\\Users\\addep\\OneDrive\\Documents\\Alex_final.pth'
alexnet_model = models.alexnet(pretrained=True)
alexnet_model.classifier[6] = nn.Linear(4096, len(test_dataset.classes))
alexnet_model.load_state_dict(torch.load(alexnet_model_path), strict=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
alexnet_model.to(device)
alexnet_model.eval()

# Evaluation function
def evaluate_model(model, dataloader):
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# Evaluate the AlexNet model on the test dataset
alexnet_model_accuracy = evaluate_model(alexnet_model, test_dataloader)

# Print the accuracy
print(f"Accuracy for AlexNet Model: {alexnet_model_accuracy:.2f}%")


In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.models as models
import matplotlib.pyplot as plt

# Set the path to the folder containing test images
test_data_dir = 'C:\\Users\\addep\\OneDrive\\Documents\\Test'

# Define the transformations applied to the input images
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the test dataset
test_dataset = ImageFolder(test_data_dir, transform=transform)

# Create the DataLoader for the test data
batch_size = 64
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Load the pre-trained AlexNet model
alexnet_model_path = ("C:\\Users\\addep\\OneDrive\\Documents\\Alex_final.pth")
alexnet_model = models.alexnet(pretrained=True)
alexnet_model.classifier[6] = nn.Linear(4096, len(test_dataset.classes))
alexnet_model.load_state_dict(torch.load(alexnet_model_path), strict=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
alexnet_model.to(device)
alexnet_model.eval()

# Evaluation function
def evaluate_model(model, dataloader):
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# Evaluate the AlexNet model on the test dataset
alexnet_model_accuracy = evaluate_model(alexnet_model, test_dataloader)

# Load the pre-trained DenseNet model
dn_model_path = "C:\\Users\\addep\\OneDrive\\Documents\\Densenet.pth"
dn_model = models.densenet201(pretrained=True)
dn_model.classifier = nn.Linear(dn_model.classifier.in_features, len(test_dataset.classes))
dn_model.load_state_dict(torch.load(dn_model_path), strict=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dn_model.to(device)
dn_model.eval()

# Evaluate the DenseNet model on the test dataset
dn_model_accuracy = evaluate_model(dn_model, test_dataloader)

# Print the accuracies
print(f"Accuracy for AlexNet Model: {alexnet_model_accuracy:.2f}%")
print(f"Accuracy for DenseNet Model: {dn_model_accuracy:.2f}%")

# Comparative graph
models = ['AlexNet', 'DenseNet']
accuracies = [alexnet_model_accuracy, dn_model_accuracy]

plt.bar(models, accuracies, color=['blue', 'green'])
plt.ylabel('Accuracy (%)')
plt.title('Comparison of AlexNet and DenseNet on Test Data')
plt.show()
